In [1]:
import tensorflow as tf

import pandas as pd
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
#from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Add, Multiply, Subtract
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, BatchNormalization
# regularizers
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from scipy import signal
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pylab as plt
import matplotlib.dates as mdates
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc
from keijzer import *
from tensorflow import keras
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dense, Input, Flatten, Add, concatenate, Dropout, Activation, Multiply, Embedding, Layer, Reshape
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D  
from keras.layers import Conv1D, AveragePooling1D, MaxPooling1D
#from keras.ops import convert_to_tensor, convert_to_numpy
#from keras.utils import plot_model
from keras import activations
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import math

class LossHistory(tensorflow.keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.rmses = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_rmses = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.rmses.append(logs.get('root_mean_squared_error'))
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_rmses.append(logs.get('val_root_mean_squared_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))



def root_squared_mean_error(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))+K.mean(K.abs(y_pred - y_true))

from keras.layers.advanced_activations import PReLU

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.experimental.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print(e)

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Import Data & Preprocessing

In [3]:
data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2020.csv'
data_source1 = pd.read_csv(data_path)
dat_source1 = pd.DataFrame(data_source1)
#dat_source1 = dat_source1.iloc[:-3,:]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2021.csv'
data_source2 = pd.read_csv(data_path)
dat_source2 = pd.DataFrame(data_source2)
dat_source2 = dat_source2.iloc[:8761,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2022.csv'
data_source3 = pd.read_csv(data_path)
dat_source3 = pd.DataFrame(data_source3)
dat_source3 = dat_source3.iloc[:8760,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2023.csv'
data_source4 = pd.read_csv(data_path)
dat_source4 = pd.DataFrame(data_source4)
dat_source4 = dat_source4.iloc[:8760,:9]

dat_arr1 = np.array(dat_source1)
dat_arr2 = np.array(dat_source2)
dat_arr3 = np.array(dat_source3)
dat_arr4 = np.array(dat_source4)

dat_arr = np.concatenate([dat_arr1, dat_arr2, dat_arr3, dat_arr4],axis=0)
dat_arr.shape # 8784, 8761, 8760, 8760

dat_source = pd.DataFrame(dat_arr, columns=['Time-Date stamp','Date','ERCOT Load','Total Wind Output','Total Wind Installed','Wind Output, % of Load','Wind Output, % of Installed','1-hr MW change','1-hr % change'])
dat_source

,Time-Date stamp,Date,ERCOT Load,Total Wind Output,Total Wind Installed,"Wind Output, % of Load","Wind Output, % of Installed",1-hr MW change,1-hr % change
0,01-01-2020 00:00:00,Jan-01,36823,12335,27040,33.5,45.617,NaN,NaN
1,01-01-2020 01:00:00,Jan-01,36511,13300,27040,36.4,49.187,965.4,7.826
2,01-01-2020 02:00:00,Jan-01,36022,13586,27040,37.7,50.245,286.2,2.152
3,01-01-2020 03:00:00,Jan-01,35790,14107,27040,39.4,52.171,520.9,3.834
4,01-01-2020 04:00:00,Jan-01,35642,14887,27040,41.8,55.055,779.6,5.527
...,...,...,...,...,...,...,...,...,...
35060,12-31-2023 20:00:00,Dec-31,44887,17161,38631,38.23,44.42,2858,19.99
35061,12-31-2023 21:00:00,Dec-31,44013,18301,38631,41.58,47.37,1139,6.64
35062,12-31-2023 22:00:00,Dec-31,43280,18607,38631,42.99,48.17,307,1.67
35063,12-31-2023 23:00:00,Dec-31,42319,17866,38631,42.22,46.25,-741,-3.98


### Dividing by Installed Capacity & Preprocessing

In [4]:
dat = pd.DataFrame()
dat['MW % change'] = np.array(dat_source.iloc[1:,3], dtype=float)-np.array(dat_source.iloc[0:-1,3], dtype=float)
dat['MW % change'] = np.divide(np.array(dat['MW % change']),np.array(dat_source.iloc[1:,4]))*100
dat['Total Wind Output'] = np.divide(np.array(dat_source.iloc[1:,3]), np.array(dat_source.iloc[1:,4]))

dat

,MW % change,Total Wind Output
0,3.56879,0.491864
1,1.05769,0.502441
2,1.92678,0.521709
3,2.88462,0.550555
4,-0.580621,0.544749
...,...,...
35059,7.3982,0.444229
35060,2.951,0.473739
35061,0.79211,0.48166
35062,-1.91815,0.462478


In [5]:
ramp_ratio = 20
load_ratio = 10
year_arr = np.concatenate([np.ones(8784)*2020, np.ones(8760)*2021, np.ones(8760)*2022, np.ones(8760)*2023])
year_df = pd.DataFrame(year_arr, columns=['Year'], dtype=int)
wind_df = pd.concat([year_df, dat], axis=1)
wind_df

,Year,MW % change,Total Wind Output
0,2020,3.56879,0.491864
1,2020,1.05769,0.502441
2,2020,1.92678,0.521709
3,2020,2.88462,0.550555
4,2020,-0.580621,0.544749
...,...,...,...
35059,2023,7.3982,0.444229
35060,2023,2.951,0.473739
35061,2023,0.79211,0.48166
35062,2023,-1.91815,0.462478


In [6]:
wind_df = wind_df.reset_index(drop=True)
std_scaler = sklearn.preprocessing.MinMaxScaler()
std_scaler_load = sklearn.preprocessing.MinMaxScaler()
std_scaler_ramp = sklearn.preprocessing.MinMaxScaler()

wind_df[['Total Wind Output']] = std_scaler.fit_transform(wind_df[['Total Wind Output']])
wind_df[['MW % change']] = std_scaler_ramp.fit_transform(wind_df[['MW % change']])
#wind_df[['Wind Change, % of Load', 'Wind Change']] = std_scaler_ramp.fit_transform(wind_df[['Wind Change, % of Load', 'Wind Change']])
wind_df = wind_df.iloc[:-1,:]
wind_df

,Year,MW % change,Total Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


In [7]:
ramp_df = pd.DataFrame(wind_df)
ramp_df.columns=['Year', 'Wind Change', 'Wind Output']
ramp_df

,Year,Wind Change,Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


## Wind Ramp Rate Forecasting

### SVD-based Feature Generation

In [8]:
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc

def create_dataset(dataset, timesteps, output_timesteps, leadtime):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps -output_timesteps - leadtime - 1):
        a = dataset[i:(i + timesteps), :]
        dataX.append(a)
        dataY.append(dataset[(i + timesteps + leadtime):(i+timesteps+output_timesteps+leadtime), :])
    return np.array(dataX), np.array(dataY)

In [9]:
norm_df = np.array(wind_df['Wind Change']).reshape(-1,1)
timesteps = 24*4
output_timesteps = 6
num_features = 1
leadtime = 3
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps

In [10]:
dec_num = timesteps
high_ind = 35

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

def svdadd(X, start, end, b_size):
    dec_x = X[start:end, :, :]
    U, S, Vh = svd(dec_x.reshape(b_size, -1))

    high_sig = np.matmul(np.matmul(U[:,:high_ind], np.diag(S[0:high_ind])), Vh[:high_ind,:])
    low_sig = X[:dec_num, :, :].reshape(b_size,-1)-high_sig

    rec_x = np.zeros((b_size, b_size, high_ind))
    #print(sum(S[0:high_ind])/sum(S))
    for i in range(high_ind):
        rec_x[:,:,i] = np.matmul((U[:,i]*S[i]).reshape(-1,1), Vh[i,:].reshape(1,-1))    
    return rec_x, low_sig

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

### Training/Test Set Division & Shuffled Learning

In [11]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)

idx = np.arange(trvaX.shape[0])
np.random.shuffle(idx)
strvaX = trvaX[idx]
strvaY = trvaY[idx]
strvaY = strvaY.reshape(-1, output_timesteps)

trvaY = trvaY.reshape(-1, output_timesteps)
teY = teY.reshape(-1, output_timesteps)

strX, svaX, strY, svaY = train_test_split(strvaX, strvaY, test_size=0.3, shuffle=False)
trX, vaX, trY, vaY = train_test_split(trvaX, trvaY, test_size=0.3, shuffle=False)

trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31461, 96, 37), (31461, 6), (3496, 96, 37), (3496, 6))

## Wind Ramp Rate Forecasting Basic Model

In [12]:
def MAEMD(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

In [13]:
from keras.layers.advanced_activations import PReLU

In [14]:
from keras import regularizers

num_features = trvaX.shape[2]
output_timesteps = trvaY.shape[1]

with tf.device('/gpu:0'):
    alpha=1.0
    gamma=1
    beta = 1
    hfilters = 168
    hkernel_size1 = 1
    hkernel_size2 = 2
    visible1 = Input(shape=(timesteps, num_features))
   
    ## Attention Block
    pera = Permute((2,1))(visible1)
    den1a = Dense(timesteps, activation='tanh')(pera)
    den1b = Dense(timesteps, activation='sigmoid')(pera)
    den1 = Multiply()([den1a, den1b])
    per2 = Permute((2,1), name='attention_vec')(den1)
    mul1 = Multiply()([visible1, per2])   
    
    ## Permuted Attention Block
    per1 = Permute((2,1))(visible1)
    den2a = Dense(num_features, activation='tanh')(visible1)
    den2b = Dense(num_features, activation='sigmoid')(visible1)
    den2 = Multiply()([den2a, den2b])
    perb2 = Permute((2,1), name='attention_vec2')(den2)
    
    mul2 = Multiply()([per1, perb2])     
    
    ## Parallel DCCNN Blocks 1-1 ~ 3-1
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=1)(mul1)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    rres01a = Add()([mul1, d1])   # (100, 25) (100, 25)
    rres01b = Subtract()([mul1, d1])
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(rres01a)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    
    d1 = Multiply()([d1a, d1b])
    rres02a = Add()([rres01a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(rres01b) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    rres02b = Subtract()([rres01b, d2])   # (100, 25) (100, 25) 
    rres02 = Concatenate()([rres02a, rres02b, rres01a, rres01b])
    #rres02 = Dropout(0.2)(rres02)
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(rres02)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    rres03a = Add()([rres02a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(rres02) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    rres03b = Subtract()([rres02b, d2])
    rres10 = Concatenate()([rres03a, rres03b, rres02])

    ## Right-half DCCNN Blocks 1-2 ~ 3-2
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=1)(mul2)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    res01a = Add()([mul2, d1])   # (100, 25) (100, 25)
    res01b = Subtract()([mul2, d1])
    
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01a)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    
    d1 = Multiply()([d1a, d1b])
    res02a = Add()([res01a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01b) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    res02b = Subtract()([res01b, d2])   # (100, 25) (100, 25) 
    res02 = Concatenate()([res02a, res02b, res01a, res01b])
    #res02 = Dropout(0.2)(res02)
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    res03a = Add()([res02a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    res03b = Subtract()([res02b, d2])
    res10 = Concatenate()([res03a, res03b, res02])

    ## Output Blocks
    out1 = Conv1D(24*7, 1, padding='same', activation=PReLU())(res10)   # 256, 11X10=110
    out1 = Dropout(0.2)(out1)  
    
    out2 = Conv1D(24*7, 1, padding='same', activation=PReLU())(rres10)   # 256, 11X10=110
    out2 = Dropout(0.2)(out2)
    
    out = Concatenate(axis=1)([out1, out2])
    
    ## Output Block
    out = Conv1D(24*7, 1, padding='same', activation=PReLU())(out) # 512,  110X5=550
    out = Dropout(0.2)(out)

    #, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
    out = GlobalAveragePooling1D()(out) 

    out = Dense(output_timesteps)(out) 
    model = Model(inputs=[visible1], outputs=[out])
    
    print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 37)]     0                                            
__________________________________________________________________________________________________
permute (Permute)               (None, 37, 96)       0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 96, 37)       1406        input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 96, 37)       1406        input_1[0][0]                    
______________________________________________________________________________________________

In [15]:
gc.collect()

1969

In [16]:
    model.compile(loss=MAEMD, optimizer='adam', metrics=['mse','mae', MAEMD])
    early_stopping =EarlyStopping(monitor='val_loss', patience=50)
    batch_size = timesteps
    epochs = 1000
    #root_squared_mean_error
    history = LossHistory()
    history.init()
    
    #hist = model.fit(trX, trY, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history, early_stopping])  # , checkpoint

In [17]:
    %%time
    b_size = timesteps
    #hist = model.fit(strvaX, strvaY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(teX, teY), callbacks=[history, early_stopping])  # , checkpoint
    hist = model.fit(strX, strY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(svaX, svaY), callbacks=[history, early_stopping])  # , checkpoint

Epoch 1/1000
230/230 [==============================] - 30s 113ms/step - loss: 0.1804 - mse: 0.0210 - mae: 0.1076 - MAEMD: 0.1805 - val_loss: 0.1488 - val_mse: 0.0169 - val_mae: 0.1020 - val_MAEMD: 0.1491
Epoch 2/1000
230/230 [==============================] - 20s 89ms/step - loss: 0.1429 - mse: 0.0156 - mae: 0.0986 - MAEMD: 0.1431 - val_loss: 0.1363 - val_mse: 0.0152 - val_mae: 0.0969 - val_MAEMD: 0.1366
Epoch 3/1000
230/230 [==============================] - 27s 116ms/step - loss: 0.1326 - mse: 0.0155 - mae: 0.0987 - MAEMD: 0.1327 - val_loss: 0.1308 - val_mse: 0.0161 - val_mae: 0.1000 - val_MAEMD: 0.1309
Epoch 4/1000
230/230 [==============================] - 23s 99ms/step - loss: 0.1269 - mse: 0.0157 - mae: 0.0995 - MAEMD: 0.1269 - val_loss: 0.1265 - val_mse: 0.0163 - val_mae: 0.1012 - val_MAEMD: 0.1266
Epoch 5/1000
230/230 [==============================] - 24s 105ms/step - loss: 0.1226 - mse: 0.0153 - mae: 0.0980 - MAEMD: 0.1226 - val_loss: 0.1240 - val_mse: 0.0155 - val_mae: 0.09

### Saving Basic Model Training Results

In [18]:
model.save('Basic Ramp Model Final_fh6.h5')

In [19]:
gc.collect()

4592

In [20]:
loss_history = hist.history['loss']
valloss_history = hist.history['val_loss']

## Basic Model Test Results

In [21]:
testPredict = model.predict(teX, batch_size=batch_size)

In [22]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [23]:
tePredict = testPredict.reshape(-1)
testY = teY.reshape(-1)
print('Error Test Score > MSE == ', npMSE(testY, tePredict), ' MAE == ', npMAE(testY, tePredict), ' MAEMS == ', npMAEMD(testY, tePredict))

Error Test Score > MSE ==  0.010124202472662495  MAE ==  0.07958302142275088  MAEMS ==  0.09609176915486234


## Wind Ramp Rate FFEL Model

In [24]:
output_timesteps = 6

In [25]:
trX, vaX, trY, vaY = train_test_split(trvaX, trvaY, test_size=1/6, shuffle=False)
trY = trY.reshape(-1,output_timesteps)
vaY = vaY.reshape(-1,output_timesteps)

In [26]:
trainPredict = model.predict(trX, batch_size=batch_size)
validPredict = model.predict(vaX, batch_size=batch_size)

In [27]:
e_tr = trainPredict - trY
e_va = validPredict - vaY
errors = np.vstack([e_tr, e_va])
prediction = np.vstack([trainPredict, validPredict])
print(errors.shape)
print(prediction.shape)

(31461, 6)
(31461, 6)


In [28]:
norm_df2 = pd.DataFrame(norm_df).iloc[:prediction.shape[0], :]
norm_df2.columns = ['Normalized Ramp']
norm_df2.head()

,Normalized Ramp
0,0.620197
1,0.544580
2,0.570751
3,0.599594
4,0.495246


In [29]:
Prindex = ['Prediction1', 'Prediction2','Prediction3','Prediction4','Prediction5','Prediction6']
Erindex = ['Error1', 'Error2','Error3','Error4','Error5','Error6']

pr_df = pd.DataFrame(prediction, columns=Prindex)
pr_df

,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6
0,0.355205,0.321475,0.440193,0.543288,0.592235,0.541352
1,0.371166,0.354698,0.365372,0.318627,0.305659,0.359306
2,0.372288,0.330047,0.252493,0.290060,0.545327,0.783457
3,0.352383,0.266103,0.291428,0.474720,0.743481,0.625581
4,0.305317,0.285573,0.512916,0.848632,0.447361,0.082599
...,...,...,...,...,...,...
31456,0.675862,0.652084,0.562497,0.450074,0.340417,0.323767
31457,0.753039,0.718695,0.574176,0.386479,0.311519,0.359828
31458,0.690589,0.590808,0.512256,0.398704,0.386623,0.370924
31459,0.565859,0.491545,0.471758,0.465640,0.393753,0.329032


In [30]:
er_df = pd.DataFrame(errors, columns=Erindex)
er_df

,Error1,Error2,Error3,Error4,Error5,Error6
0,-0.086252,-0.101718,0.059875,0.143259,0.267488,0.191326
1,-0.052027,-0.025619,-0.034657,-0.006119,-0.044368,-0.024353
2,-0.008030,-0.069982,-0.072253,-0.059966,0.161669,-0.090428
3,-0.047646,-0.058644,-0.058599,0.091061,-0.130404,0.099376
4,-0.019430,-0.064454,0.129258,-0.025253,-0.078844,-0.037905
...,...,...,...,...,...,...
31456,0.005838,-0.030489,-0.041748,-0.030126,-0.000392,0.030513
31457,0.070467,0.114450,0.093976,0.045670,0.018266,-0.075454
31458,0.086343,0.110608,0.171447,0.105451,-0.048659,-0.336826
31459,0.085659,0.150736,0.178505,0.030359,-0.313997,-0.257470


In [31]:
norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2.head()

,Normalized Ramp,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6,Error1,Error2,Error3,Error4,Error5,Error6
0,0.620197,0.355205,0.321475,0.440193,0.543288,0.592235,0.541352,-0.086252,-0.101718,0.059875,0.143259,0.267488,0.191326
1,0.544580,0.371166,0.354698,0.365372,0.318627,0.305659,0.359306,-0.052027,-0.025619,-0.034657,-0.006119,-0.044368,-0.024353
2,0.570751,0.372288,0.330047,0.252493,0.290060,0.545327,0.783457,-0.008030,-0.069982,-0.072253,-0.059966,0.161669,-0.090428
3,0.599594,0.352383,0.266103,0.291428,0.474720,0.743481,0.625581,-0.047646,-0.058644,-0.058599,0.091061,-0.130404,0.099376
4,0.495246,0.305317,0.285573,0.512916,0.848632,0.447361,0.082599,-0.019430,-0.064454,0.129258,-0.025253,-0.078844,-0.037905


In [32]:
norm_df22 = pd.DataFrame(norm_df).iloc[prediction.shape[0]+timesteps:, :]
norm_df22.columns = ['Normalized Wind']
npnorm22 = np.array(norm_df22)

In [33]:
pr_df = pd.DataFrame(tePredict.reshape(-1,6))
pr_df

,0,1,2,3,4,5
0,0.362638,0.360067,0.320813,0.358743,0.406330,0.442248
1,0.458406,0.425269,0.369052,0.373581,0.412202,0.456250
2,0.415771,0.416348,0.396263,0.406548,0.428573,0.460973
3,0.389874,0.368727,0.373699,0.401948,0.438417,0.636120
4,0.278604,0.333515,0.366870,0.446477,0.623106,0.448852
...,...,...,...,...,...,...
3491,0.609953,0.652093,0.609014,0.591178,0.509967,0.415477
3492,0.576092,0.561833,0.552108,0.615220,0.546546,0.490200
3493,0.478875,0.582181,0.570662,0.505341,0.481847,0.601246
3494,0.549657,0.616933,0.569664,0.572226,0.595581,0.572503


In [34]:
teY = testY.reshape(-1,6)
e_te = testPredict-teY
er_df = pd.DataFrame(e_te, columns=Erindex)
er_df

,Error1,Error2,Error3,Error4,Error5,Error6
0,0.069385,-0.075215,-0.386937,-0.227759,-0.038543,-0.059133
1,0.023124,-0.282480,-0.217450,-0.071292,-0.089178,-0.093645
2,-0.291979,-0.170154,-0.048609,-0.094833,-0.121323,0.023613
3,-0.196627,-0.076146,-0.127682,-0.147947,0.001058,0.148966
4,-0.166269,-0.167866,-0.183026,0.009117,0.135952,0.095814
...,...,...,...,...,...,...
3491,0.014830,0.178416,0.133544,0.051166,-0.043063,-0.165694
3492,0.102415,0.086364,0.012096,0.062190,-0.034625,-0.090658
3493,0.003405,0.042169,0.017632,-0.075830,-0.099011,-0.173396
3494,0.009644,0.063903,-0.011506,-0.008633,-0.179062,-0.163009


In [35]:
prnorm = np.array(pr_df)
ernorm =np.array(er_df)
print(ernorm.shape)

(3496, 6)


In [36]:
etedat = np.concatenate((npnorm22[:prnorm.shape[0],:], prnorm, ernorm), axis=1)
#etedat[169:171,:]

In [37]:
eteX, eteY = create_dataset(etedat, timesteps, 1, 0)
eteY.shape

(3398, 1, 13)

In [38]:
eteY = eteY[:,:,-6:].reshape(-1,6)

In [39]:
norm_df2 = np.array(norm_df2)
output_timesteps = 1
Xe, Ye = create_dataset(norm_df2, timesteps, 1, 0)
Ye.shape

(31363, 1, 13)

In [40]:
Ye = Ye[:,:,-6:].reshape(-1, 6)
print(Xe.shape)
print(Ye.shape)

(31363, 96, 13)
(31363, 6)


In [41]:
trXe, vaXe, trYe, vaYe = train_test_split(Xe, Ye, test_size=0.3, shuffle=False)

In [42]:
gc.collect()

6285

In [43]:
eteX.shape, eteY.shape, Xe.shape, Ye.shape

((3398, 96, 13), (3398, 6), (31363, 96, 13), (31363, 6))

In [44]:
def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))*10+K.mean(K.abs(y_pred - y_true))

In [45]:
num_features = Xe.shape[2]

with tf.device('/gpu:0'):
    alpha=1.0
    gamma=1.2
    beta = 1
    hfilters = 256
    hkernel_size1 = 1
    hkernel_size2 = 2
    visible1e = Input(shape=(timesteps, num_features))
   
    per1e = Permute((2,1))(visible1e)
    den1ae = Dense(timesteps, activation='tanh')(per1e)
    den1be = Dense(timesteps, activation='sigmoid')(per1e)
    den1e = Multiply()([den1ae, den1be])
    per2e = Permute((2,1), name='attention_vec')(den1e)
    mul1e = Multiply()([visible1e, per2e])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=1)(mul1e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    res01ae = Add()([visible1e, d1e])   # (100, 25) (100, 25)
    res01be = Subtract()([visible1e, d1e])

    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01ae)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    
    d1e = Multiply()([d1ae, d1be])
    res02ae = Add()([res01ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01be) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    res02be = Subtract()([res01be, d2e])   # (100, 25) (100, 25) 
    res02e = Concatenate()([res02ae, res02be, res01ae, res01be])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=8)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    res03ae = Add()([res02ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=8)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    res03be = Subtract()([res02be, d2e])   # (100, 25) (100, 25)
    res03e = Concatenate()([res03ae, res03be, res02e])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=8)(res03e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    res04ae = Add()([res03ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=8)(res03e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    res04be = Subtract()([res02be, d2e])   # (100, 25) (100, 25)
    res04e = Concatenate()([res04ae, res04be, res03e])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res04e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    res05ae = Add()([res04ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res04e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    res05be = Subtract()([res04be, d2e])   # (100, 25) (100, 25)
    res05e = Concatenate()([res05ae, res05be, res04e])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res05e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=1)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])

    res06ae = Add()([res05ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res05e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=1)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])

    res06be = Subtract()([res05be, d2e])   # (100, 25) (100, 25)
    res06e = Concatenate()([res05ae, res05be])
    
    res10e = Concatenate()([res02e, res03e, res04e, res05e, res06e])   # 
    
    #print('res10 :', res10.shape)  # (None, 24, 11) 
    
    oute = Conv1D(timesteps*5, 1, padding='same', activation=PReLU())(res10e)   # 256, 11X10=110
    oute = Dropout(0.2)(oute)   #SpatialDropout1D
    
    oute = Conv1D(timesteps*3, 1, padding='same', activation=PReLU())(oute) # 512,  110X5=550
    oute = Dropout(0.2)(oute)
    
    oute = GlobalAveragePooling1D()(oute) # pool_size=2, strides=1
    
    oute = Dense(6)(oute) 
    modele = Model(inputs=[visible1e], outputs=[oute])
    
    print(modele.summary())
    
    modele.compile(loss=mse_mae, optimizer='adam', metrics=['mse','mae','mape'])
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    batch_size = 168
    epochs = 1000

    history_e = LossHistory()
    history_e.init()
    
    #hist = model.fit(trX, trY, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history, early_stopping])  # , checkpoint

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 96, 13)]     0                                            
__________________________________________________________________________________________________
permute_2 (Permute)             (None, 13, 96)       0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 13, 96)       9312        permute_2[0][0]                  
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 13, 96)       9312        permute_2[0][0]                  
____________________________________________________________________________________________

In [46]:
gc.collect()

2538

In [47]:
# %%time
    histe = modele.fit(trXe, trYe, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(vaXe, vaYe), callbacks=[history_e, early_stopping])  # , checkpoint

Epoch 1/1000
229/229 [==============================] - 59s 220ms/step - loss: 0.1534 - mse: 0.0090 - mae: 0.0637 - mape: 481.0083 - val_loss: 0.0952 - val_mse: 0.0044 - val_mae: 0.0508 - val_mape: 786.1519
Epoch 2/1000
229/229 [==============================] - 42s 181ms/step - loss: 0.1055 - mse: 0.0051 - mae: 0.0541 - mape: 378.7788 - val_loss: 0.0839 - val_mse: 0.0037 - val_mae: 0.0466 - val_mape: 722.4144
Epoch 3/1000
229/229 [==============================] - 42s 182ms/step - loss: 0.0901 - mse: 0.0041 - mae: 0.0487 - mape: 390.9835 - val_loss: 0.0715 - val_mse: 0.0030 - val_mae: 0.0418 - val_mape: 484.3049
Epoch 4/1000
229/229 [==============================] - 40s 175ms/step - loss: 0.0810 - mse: 0.0036 - mae: 0.0454 - mape: 414.9786 - val_loss: 0.0686 - val_mse: 0.0028 - val_mae: 0.0406 - val_mape: 695.7495
Epoch 5/1000
229/229 [==============================] - 43s 190ms/step - loss: 0.0779 - mse: 0.0034 - mae: 0.0442 - mape: 378.9064 - val_loss: 0.0677 - val_mse: 0.0027 - va

### Saving FFEL Model Training Results

In [48]:
eloss_history = histe.history['loss']
valeloss_history = histe.history['val_loss']

In [49]:
#np.savetxt('ramp_elosshistory.txt',(eloss_history, valeloss_history))

In [50]:
modele.save('Error Learning Ramp Model_fh6.h5')

In [51]:
gc.collect()

4570

## FFEL Model Test Results

In [52]:
trainePredict = modele.predict(Xe, batch_size=batch_size)
etePredict = modele.predict(eteX, batch_size=batch_size)

In [53]:
trePredict = trainePredict.reshape([-1])
trainYe = Ye.reshape([-1])

print('Error Training Score > MSE == ', (np.mean(np.square(trainYe-trePredict))), ' MAE == ', mean_absolute_error(trainYe,trePredict), ' RMSE == ', np.sqrt(np.mean(np.square(trainYe-trePredict))))

Error Training Score > MSE ==  0.0027508268982510123  MAE ==  0.04010213203201518  RMSE ==  0.052448325981398224


In [54]:
etestPredict = etePredict.reshape([-1])
testYe = eteY.reshape([-1])

print('Error Training Score > MSE == ', (np.mean(np.square(testYe-etestPredict))), ' MAE == ', mean_absolute_error(testYe,etestPredict), ' RMSE == ', np.sqrt(np.mean(np.square(testYe-etestPredict))))

Error Training Score > MSE ==  0.004184260277979722  MAE ==  0.05045028076227565  RMSE ==  0.06468585840799922


## Final Prediction Results

In [55]:
testPredict = tePredict.reshape(-1,6)
addtestPredict = -etePredict + testPredict[timesteps:-2,:]
print(addtestPredict.shape)

(3398, 6)


In [56]:
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-testPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), 'MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]))
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-addtestPredict))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], addtestPredict), 'MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], addtestPredict))

Error Test Score > MSE ==  0.010049596148361997  MAE ==  0.0792748119060706 MAPE ==  15.831536508159912
Error Test Score > MSE ==  0.0041842602580344345  MAE ==  0.05045028057592434 MAPE ==  10.55787095438928
